In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embedding_function = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
from langchain.schema import Document
import json

C:\Users\Anandhu\AppData\Local\Temp\ipykernel_16928\2754882615.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
c:\Users\Anandhu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h2>LOADING THE SAVED CHUNKS AND THE CHROMA VECTOR DATABASE</h2>

In [2]:
def load_chunks(filename="preprocessed_chunks.json"):
    with open(filename, 'r') as f:
        chunk_dicts = json.load(f)
    return [Document(page_content=c["page_content"], metadata=c["metadata"]) for c in chunk_dicts]

# Load chunks instead of reprocessing PDF
chunks = load_chunks()

# Load the stored vector database
vectorstore = Chroma(persist_directory="./chroma_db_pedition", embedding_function=embedding_function)

C:\Users\Anandhu\AppData\Local\Temp\ipykernel_16928\4196027506.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./chroma_db_pedition", embedding_function=embedding_function)


<h2>USING ONLY VECTOR SIMILARITY</h2>

In [3]:
# Retrieve the top 3 most similar chunks for a user query
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
relevant_chunks = retriever.get_relevant_documents("What challenge did Penelope use to test the suitors, and how did Odysseus prove his identity?")


''' 
Penelope devised a challenge where the suitors had to string Odysseus' great bow and shoot an arrow through twelve axe heads. 
None of the suitors could accomplish the feat. However, when Odysseus, disguised as a beggar, attempted the challenge, 
he easily strung the bow and shot the arrow through all twelve axe heads, proving his true identity. 
'''

C:\Users\Anandhu\AppData\Local\Temp\ipykernel_16928\612966275.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_chunks = retriever.get_relevant_documents("What challenge did Penelope use to test the suitors, and how did Odysseus prove his identity?")


" \nPenelope devised a challenge where the suitors had to string Odysseus' great bow and shoot an arrow through twelve axe heads. \nNone of the suitors could accomplish the feat. However, when Odysseus, disguised as a beggar, attempted the challenge, \nhe easily strung the bow and shot the arrow through all twelve axe heads, proving his true identity. \n"

In [4]:
for chunk in relevant_chunks:
    print(chunk.page_content)  # Shows the most relevant textbook sections

but he laid his hand on my mouth, and in the fulness of his wisdom suffered me not to speak. But come with me and I will stake my life on it; and if I play thee false, do thou slay me by a death most pitiful.” Then wise Penelope made answer to her: “Dear nurse, it is hard for thee, how wise soever, to observe the purposes of the everlasting gods. None the less let us go to my child, that I may see the wooers dead, and him that slew them.” With that word she went down from the upper chamber, and much her heart debated, whether she should stand apart, and question her dear lord or draw nigh, and clasp and kiss his head and hands. But when she had come within and had crossed the threshold of stone, she sat down over against Odysseus, in the light of the fire, by the further wall. Now he was sitting by the tall pillar, looking down and waiting to know if perchance his noble wife would speak to him, when her eyes beheld him. But she sat long in silence, and amazement came upon her soul, and

In [5]:
relevant_chunks = retriever.get_relevant_documents("who wrote odyssey?")
for chunk in relevant_chunks:
    print(chunk.page_content)

The Odyssey
The Odyssey
The Project Gutenberg EBook of The Odyssey, by Homer This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you'll have to check the laws of the country where you are located before using this ebook. Title: The Odyssey Author: Homer Translator: Butcher & Lang Release Date: April, 1999 [EBook #1728] Last updated: April 16, 2020 Language: English *** START OF THIS PROJECT GUTENBERG EBOOK THE ODYSSEY *** Produced by Jim Tinsley


In [6]:
relevant_chunks = retriever.get_relevant_documents("who wrote this book?")
for chunk in relevant_chunks:
    print(chunk.page_content)

THE FULL PROJECT GUTENBERG LICENSE PLEASE READ THIS BEFORE YOU DISTRIBUTE OR USE THIS WORK To protect the Project Gutenberg-tm mission of promoting the free distribution of electronic works, by using or distributing this work (or any other work associated in any way with the phrase "Project Gutenberg"), you agree to comply with all the terms of the Full Project Gutenberg-tm License available with this file or online at www.gutenberg.org/license. Section 1. General Terms of Use and Redistributing Project Gutenberg-tm electronic works 1.A. By reading or using any part of this Project Gutenberg-tm electronic work, you indicate that you have read, understand, agree to and accept all the terms of this license and intellectual property (trademark/copyright) agreement. If you do not agree to abide by all the terms of this agreement, you must cease using and return or destroy all copies of Project Gutenberg-tm electronic works in your possession. If you paid a fee for obtaining a copy of or ac

<h2>USING BOTH VECTOR SIMILARITY AND KEYWORD SIMILARITY</h2>

In [7]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Existing vector retriever
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Keyword retriever (BM25)
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 5

# Hybrid ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.7, 0.3]  # Tune based on your data
)

relevant_chunks = ensemble_retriever.get_relevant_documents("who wrote odyssey?")

In [8]:
relevant_chunks

[Document(metadata={'page': 2, 'page_label': '3', 'source': 'The-Odyssey.pdf'}, page_content='The Odyssey'),
 Document(metadata={'page': 1, 'page_label': '2', 'source': 'The-Odyssey.pdf'}, page_content="The Project Gutenberg EBook of The Odyssey, by Homer This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you'll have to check the laws of the country where you are located before using this ebook. Title: The Odyssey Author: Homer Translator: Butcher & Lang Release Date: April, 1999 [EBook #1728] Last updated: April 16, 2020 Language: English *** START OF THIS PROJECT GUTENBERG EBOOK THE ODYSSEY *** Produced by Jim Tinsley"),
 Document(metadata={'page': 3, 'page_label': '4', 'source': 'The-Odysse

<h2>HYBRID APPROACH OF USING HYBRID RETRIEVAL APPROACH FOLLOWED BY RE-RANKING</h2>

uses a pre-trained model (BAAI/bge-reranker-large) that’s already trained to judge question-chunk relevance.
bge-reranker-large is a cross encoder.


<span style = "font-weight: bold; font-size: 23px;">1. Hybrid Retrieval</span>( code combines two retrieval methods )


<span style = "font-weight: bold; font-size: 15px;">Vector Search (Semantic Search) :</span></br>
Uses embeddings (e.g., intfloat/e5-large-v2) to find chunks that are semantically similar to the query.


<span style = "font-weight: bold; font-size: 15px;">Keyword Search (BM25) :</span></br>
Uses exact term matching to find chunks containing keywords like "hospitality," "themes," or "Odyssey."


The EnsembleRetriever combines the results from both methods using weights , giving more importance to vector search and less to keyword search.



<span style = "font-weight: bold; font-size: 23px;">2. Re-Ranking</span>


After retrieving chunks using the hybrid approach, code uses a cross-encoder re-ranker (BAAI/bge-reranker-large) to :</br>
Score each chunk based on how well it answers the query.</br>
Sort the chunks by relevance and keep only the top 3.</br>

<span style = "font-size: 14px;">
A cross-encoder reranker (like BAAI/bge-reranker-large) does not need to see data beforehand. </br>
Instead, it works based on general language understanding and semantic similarity.</br>
The model was trained on massive amounts of text (e.g., Wikipedia, books, academic papers).</br>
It learned how to compare text pairs and decide if one is relevant to the other.</br>
It does not need to be fine-tuned on The Odyssey specifically—it just compares question to the text using its existing knowledge of language.</br>
When passing query and retrieved document chunks to BAAI/bge-reranker-large, the model compares them in real-time using semantic understanding.</span></br></br>

<span style = "font-weight: bold; font-size: 23px;">Scoring</span>

<span style = "font-weight: bold;">pairs = [[query, chunk.page_content] for chunk in chunks]</span></br>
It creates a list of query-chunk pairs</br></br>
<span style = "font-weight: bold;font-size : 13px;">output : 
[</br>
    <span style = "margin-left: 60px;">["Who is Odysseus?", "Odysseus is the king of Ithaca and a hero of the Trojan War."]</span>,</br>
    <span style = "margin-left: 60px;">["Who is Odysseus?", "Penelope waited 20 years for her husband's return."]</span>,</br>
    <span style = "margin-left: 60px;">["Who is Odysseus?", "Odysseus is famous for his intelligence and cunning."]</span></br>
]
</span>

The cross-encoder model takes two inputs at a time (the query and a document) and determines how relevant they are.</br></br></br></br>
<span style = "font-weight: bold;">Tokenizer : </span>converts the text into numerical tokens that the model understands.</br>
<span style = "font-weight: bold;">padding=True : </span>Ensures all inputs are the same length.</br>
<span style = "font-weight: bold;">truncation=True : </span>Cuts long text at max_length=512 tokens (to fit model constraints).</br>
<span style = "font-weight: bold;">return_tensors="pt" : </span>Converts the text into PyTorch tensors for model input.</br>

<span style = "font-size: 14px;">
Vectors are just a type of tensor (specifically a 1D tensor).</br>
In retrieval, embeddings (which are vectors) are used to represent the semantic meaning of text.</br>
Tensors are used throughout deep learning models for efficient computation.</br>
Cross-encoders use tensors to process and compare pairs of text (e.g., query + chunk) and compute similarity based on semantic meaning.</span></br>

<span style = "font-weight: bold;font-size : 13px;">Example of Tokenized Output : </br>
{</br>
    <span style = "margin-left: 60px;">"input_ids": tensor([[ 101,  2040,  2003,  ...,  102,  ...,  1037, 2034,  ..., 102]]),</span>,</br>
    <span style = "margin-left: 60px;">"attention_mask": tensor([[1, 1, 1, ..., 1, ..., 1, 1, ..., 1]])</span>,</br>
}
</span>
</br></br></br>

<span style = "font-weight: bold;">Getting Model Predictions (Relevance Scores)</span></br>
with torch.no_grad():</br>
    <span style = "margin-left: 60px;">scores = model(**inputs).logits.view(-1).float()</span>

model(**inputs) → Runs the bge-reranker-large model on the tokenized input.</br>
.logits.view(-1).float() → Extracts the relevance scores for each query-chunk pair.</br>

<span style = "font-weight: bold;font-size : 13px;">Example of Model Scores:
scores = tensor([3.1, 1.5, 2.7])  <span style = "font-weight: bold;font-size : 11px;"># Higher scores mean better relevance</span>
</span>
</br></br></br>

<span style = "font-weight: bold;">probabilities = torch.sigmoid(scores).numpy()</span></br>
<span style = "font-weight: bold;font-size : 13px;">
Example Conversion :</br>
raw_scores = [3.1, 1.5, 2.7]  <span style = "font-weight: bold;font-size : 11px;"># Model output</span></br>
probabilities = [0.96, 0.82, 0.93]  <span style = "font-weight: bold;font-size : 11px;"># After applying sigmoid</span>
</span>
</br></br></br>

<span style = "font-weight: bold;">return float(np.max(probabilities)) :</span></br> Returning the Highest Confidence Score

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np



# Load cross-encoder model
rerank_model_name = "BAAI/bge-reranker-large"
tokenizer = AutoTokenizer.from_pretrained(rerank_model_name)
model = AutoModelForSequenceClassification.from_pretrained(rerank_model_name)



# BELOW PART RETURNS WITHOUT CONFIDENT SCORES
# def rerank_chunks(query, chunks, top_k=3):
#     pairs = [[query, chunk.page_content] for chunk in chunks]
#     inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors="pt", max_length=512)
#     with torch.no_grad():
#         scores = model(**inputs).logits.view(-1).float()
#     sorted_indices = scores.argsort(descending=True)
#     return [chunks[i] for i in sorted_indices[:top_k]]



# FUNCTION WHICH ALSO RETURNS CONFIDENCE ALONG WITH RERANKING
def rerank_chunks(query, chunks, top_k=3):
    pairs = [[query, chunk.page_content] for chunk in chunks]
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors="pt", max_length=512)

    with torch.no_grad():
        scores = model(**inputs).logits.view(-1).float()

    sorted_indices = scores.argsort(descending=True)
    top_chunks = [chunks[i] for i in sorted_indices[:top_k]]
    top_scores = scores[sorted_indices[:top_k]].numpy()  # Extract scores for top results

    return top_chunks, top_scores  # Return ranked chunks + scores



def calculate_confidence(scores):
    """Takes the scores from `rerank_chunks` and applies sigmoid to get a confidence score."""
    probabilities = torch.sigmoid(torch.tensor(scores)).numpy()
    return float(np.max(probabilities))  # Return the highest confidence score

In [10]:
initial_chunks = ensemble_retriever.get_relevant_documents("who wrote odyssey?")
final_chunks, relevance_scores = rerank_chunks("who wrote odyssey?", initial_chunks, top_k=3)

confidence = calculate_confidence(relevance_scores)
final_chunks

# 16.1 SECONDS
# 21.9 SECONDS SECOND RUN

[Document(metadata={'page': 1, 'page_label': '2', 'source': 'The-Odyssey.pdf'}, page_content="The Project Gutenberg EBook of The Odyssey, by Homer This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you'll have to check the laws of the country where you are located before using this ebook. Title: The Odyssey Author: Homer Translator: Butcher & Lang Release Date: April, 1999 [EBook #1728] Last updated: April 16, 2020 Language: English *** START OF THIS PROJECT GUTENBERG EBOOK THE ODYSSEY *** Produced by Jim Tinsley"),
 Document(metadata={'page': 2, 'page_label': '3', 'source': 'The-Odyssey.pdf'}, page_content='The Odyssey'),
 Document(metadata={'page': 3, 'page_label': '4', 'source': 'The-Odysse

In [11]:
initial_chunks = ensemble_retriever.get_relevant_documents("Explain the themes of hospitality in The Odyssey.")
final_chunks, relevance_scores = rerank_chunks("Explain the themes of hospitality in The Odyssey.", initial_chunks, top_k=3)
final_chunks

# 18.0 SECONDS
# 18.4 SECONDS SECOND RUN

[Document(metadata={'page': 2, 'page_label': '3', 'source': 'The-Odyssey.pdf'}, page_content='The Odyssey'),
 Document(metadata={'page': 163, 'page_label': '164', 'source': 'The-Odyssey.pdf'}, page_content='minstrel Demodocus, whom the people honoured. But Odysseus would ever turn his head toward the splendour of the sun, as one fain to hasten his setting: for verily he was most eager to return. And as when a man longs for his supper, for whom all day long two dark oxen drag through the fallow field the jointed plough, yea and welcome to such an one the sunlight sinketh, that so he may get him to supper, for his knees wax faint by the way, even so welcome was the sinking of the sunlight to Odysseus. Then straight he spake among the Phaeacians, masters of the oar, and to Alcinous in chief he made known his word, saying: “My lord Alcinous, most notable of all the people, pour ye the drink offering, and send me safe upon my way, and as for you, fare ye well. For now have I all that my hea

<h2>LOADING LLAMA3</h2>

In [12]:
from langchain_groq import ChatGroq
from AI_GATEWAYS import groq_api_key

llm = ChatGroq(
    # model_name = "deepseek-r1-distill-llama-70b",
    model_name = "llama3-70b-8192",
    # model_name = "mixtral-8x7b-32768",
    temperature=0,
    groq_api_key = groq_api_key
)

<h2>LLM IMPLEMENTATION WITHOUT MEMORY AND THRESHHOLD</h2>

In [13]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser

# # Custom prompt template for QA
# qa_prompt = ChatPromptTemplate.from_template(
#     """Use the following context from Homer's Odyssey to answer the question. 
#     If you don't know the answer, say you don't know. Be precise and stay true to the text.
    
#     Context:
#     {context}
    
#     Question: {question}
    
#     Answer in complete sentences, citing text evidence:"""
# )

# # Create processing chain
# qa_chain = (
#     {"context": lambda x: format_chunks(x["chunks"]), "question": lambda x: x["question"]}
#     | qa_prompt
#     | llm
#     | StrOutputParser()
# )

# def format_chunks(chunks):
#     return "\n\n".join([f"Page {c.metadata['page']}: {c.page_content}" for c in chunks])

In [14]:
# def ask_question(question):
#     # Retrieve relevant chunks
#     initial_chunks = ensemble_retriever.get_relevant_documents(question)
#     final_chunks, relevance_scores = rerank_chunks(question, initial_chunks, top_k=3)
    
#     # Generate answer
#     answer = qa_chain.invoke({"question": question, "chunks": final_chunks})
    
#     # Add sources
#     sources = list(set(c.metadata["page"] for c in final_chunks))
#     return f"{answer}\n\nSources: Pages {', '.join(map(str, sources))}"

# # Example usage
# print(ask_question("What challenge did Penelope use to test the suitors?"))

<h2>LLM IMPLEMENTATION USING MEMORY AND THRESHHOLD</h2>

<span style = "font-weight: bold;">k=3 :</span> Retains the last 3 exchanges

<span style = "font-weight: bold;">memory_key :</span> The key where conversation history is stored

<span style = "font-weight: bold;">return_messages = True :</span> Returns full messages instead of just text

<span style = "font-weight: bold;">output_key="answer" :</span> Ensures responses are linked correctly

Both <span style = "color: blue;">chat_history</span> and <span style = "color: blue;">answer</span> are custom names that you define when setting up memory and output storage. They are not predefined by LangChain.
stored conversation history will be saved under the key <span style = "color: blue;">"chat_history"</span>.
<span style = "color: blue;">output_key="answer"</span> tells LangChain where to store the LLM’s response.



</br></br><span style = "font-weight: bold;color: red;">qa_chain part creates a pipeline where the user's question and retrieved chunks are sequentially processed before being sent to the LLM for answering.</span>

<span style = "font-weight: bold;">"context"</span> → Retrieved chunks (relevant text from The Odyssey)

<span style = "font-weight: bold;">"question"</span> → The user’s question

<span style = "font-weight: bold;">"chat_history"</span> → The conversation memory (last 3 exchanges)


<span style = "font-weight: bold;">| qa_prompt :</span></br> formats the inputs into a structured prompt using ChatPromptTemplate
The qa_prompt fills in {chat_history}, {context}, and {question} with actual values.


<span style = "font-weight: bold;">| llm :</span></br> Passes the formatted prompt to the LLM (llama3-70b-8192).
The LLM reads the input, processes the question in context, and generates a coherent answer.


<span style = "font-weight: bold;">| StrOutputParser() :</span></br> Converts the raw LLM output into a clean string.
The LLM response might be wrapped in a structured format (like JSON).
StrOutputParser() extracts only the raw text, making it easier to display or process further.

<span style = "font-weight: bold; color:red; font-size:12px;">EG:</span>
<span style = "font-weight: bold;font-size:14px;">llm_output :</span>
<span style = "font-size:14px;">"text": "Penelope challenged the suitors to string Odysseus' bow and shoot an arrow through twelve axe heads. Only Odysseus, in disguise, was able to complete the challenge."</span>

<span style = "font-weight: bold;font-size:14px;">StrOutputParser() output :</span>
<span style = "font-size:14px;">Penelope challenged the suitors to string Odysseus' bow and shoot an arrow through twelve axe heads. Only Odysseus, in disguise, was able to complete the challenge.</span>


In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# Create memory that retains last 3 exchanges
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)


# Modified QA Chain with Memory
qa_prompt = ChatPromptTemplate.from_template(
    """Answer based on our conversation history and the Odyssey context:
    
    Chat History:
    {chat_history}
    
    Context:
    {context}
    
    Question: {question}
    
    Answer in complete sentences, citing text evidence. 
    If unsure, say so:"""
)


qa_chain = (
    {"context": lambda x: x["chunks"], 
     "question": lambda x: x["question"],
     "chat_history": lambda x: x["chat_history"]}
    | qa_prompt
    | llm
    | StrOutputParser()
)

C:\Users\Anandhu\AppData\Local\Temp\ipykernel_16928\3799967650.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


<span style = "font-weight: bold;">format_chunks : </span></br>
This function formats retrieved text by displaying the page number and content clearly.
Helps the LLM generate contextually aware responses.

<span style = "font-size:13px;">
<span style = "color:red;">EG : </span></br>
<span style = "font-weight: bold;">input : </span></br>
chunks = [</br>
    <span style = "margin-left: 90px;">Document(page_content="Odysseus returned home after 20 years of war and struggle.", metadata={"page": 120}),</span></br>
    <span style = "margin-left: 90px;">Document(page_content="Penelope devised a test using Odysseus' bow to challenge the suitors.", metadata={"page": 132}),</span></br>
    <span style = "margin-left: 90px;">Document(page_content="Only Odysseus was able to string the bow, proving his identity.", metadata={"page": 135})</span></br>
]

<span style = "font-weight: bold;">Output : </span></br>
Page 120: Odysseus returned home after 20 years of war and struggle.</br>
Page 132: Penelope devised a test using Odysseus' bow to challenge the suitors.</br>
Page 135: Only Odysseus was able to string the bow, proving his identity.
</span></br></br></br>

In [16]:
def format_chunks(chunks):
    return "\n\n".join([f"Page {c.metadata['page']}: {c.page_content}" for c in chunks])



def get_threshold_response():
    """Response when confidence is below threshold"""
    return "I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?"



# Modified ask_question function with confidence scoring
def ask_question(question, confidence_threshold=0.65):
    # Retrieve context
    initial_chunks = ensemble_retriever.get_relevant_documents(question)
    final_chunks, relevance_scores = rerank_chunks(question, initial_chunks, top_k=3)
    
    # Calculate confidence
    confidence = calculate_confidence(relevance_scores)
    
    # Generate answer
    answer = qa_chain.invoke({
        "question": question,
        "chunks": format_chunks(final_chunks),
        "chat_history": memory.load_memory_variables({})["chat_history"]
    })
    
    # Store interaction in memory
    memory.save_context({"question": question}, {"answer": answer})
    
    # Add confidence and sources
    sources = list(set(c.metadata["page"] for c in final_chunks))
    response = f"{answer}\n\nConfidence: {confidence:.0%}\nSources: Pages {', '.join(map(str, sources))}"
    
    return response if confidence >= confidence_threshold else f"{get_threshold_response()}\n\n{response}"

In [17]:
# Start conversation
print(ask_question("Who is Telemachus?"))
# Output includes answer + "Confidence: 82%"

I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?

Telemachus is the noble son of the great-hearted Odysseus, as stated on page 184: "Pallas Athene went to the wide land of Lacedaemon, to put the noble son of the great-hearted Odysseus in mind of his return, and to make him hasten his coming." This indicates that Telemachus is the son of Odysseus, the king of Ithaca. Additionally, on page 195, Telemachus is referred to as "dear child" by Theoclymenus, further emphasizing his relationship to Odysseus.

Confidence: 1%
Sources: Pages 184, 41, 195


In [18]:
# Follow-up question
print(ask_question("And what about his relationship with Odysseus?"))
# The memory maintains context about Telemachus

I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?

Telemachus is the son of Odysseus, as stated on page 184, and is referred to as "dear child" by Theoclymenus on page 195, emphasizing his relationship to Odysseus. Additionally, on page 292, Odysseus sees his father, Laertes, and considers revealing his identity to him, indicating a close family relationship. Furthermore, on page 175, Telemachus expresses his longing for Odysseus, calling him "worshipful" and stating that he loved and cared for him exceedingly, demonstrating a strong bond between them.

Confidence: 5%
Sources: Pages 292, 230, 175


In [19]:
# Low-confidence response
print(ask_question("Did Odysseus use a lightsaber?"))
# Triggers threshold response with lower confidence score

No, Odysseus did not use a lightsaber. There is no mention of a lightsaber in the provided text from the Odyssey. In fact, the text describes Odysseus using a bronze sword (Page 268), a bow and arrows (Page 268), and a spear (Page 268), but there is no mention of a lightsaber.

Confidence: 99%
Sources: Pages 224, 268, 79
